# First order ZV-MCMC estimates

Ce notebook répond à **deuxième question** du projet.

Avec la méthode Ordinary-MCMC, nous avons déterminé l'estimateur en approximant l'espérance du vecteur de paramètres cible $ \boldsymbol{\omega} = (\omega_1, \omega_2, \omega_3) $ en fonction des données par une moyenne empirique sur 2000 itérations.

La méthode Zero-Variance Markov Chain Monte Carlo (ZV-MCMC) consiste à réduire l'erreur MCMC en remplaçant directement $ \boldsymbol{\omega} $ par un vecteur $ \tilde{\boldsymbol{\omega}} $ obtenu en rénormalisant correctement $ \boldsymbol{\omega} $. En suivant les directives de l'article, nous introduisons un polynôme de premier degré pour construire $ \tilde{\boldsymbol{\omega}} $, de telle sorte que son espérance soit égale à $ \mathbb{E}[\boldsymbol{\omega}] $, mais sa variance soit beaucoup plus petite.

**Procédure ZV-MCMC :**

- *Première étape* : Utilisation d'une courte simulation MCMC (2000 itérations) pour estimer les coefficients des variables de contrôle.

- *Deuxième étape* : Utilisation d'une longue simulation MCMC (10000 itérations) pour estimer la moyenne postérieure de chaque paramètre sous la forme rénormalisée en introduisant les variables de contrôle.


## Importation des librairies  

In [ ]:
! pip install import-ipynb
! pip install ipynb
! pip install arch
! pip install numpy 
! pip install matplotlib
! pip install scikit-learn

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from arch.univariate import GARCH, ZeroMean
import pandas as pd 
from math import log 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import statsmodels.api as sm
import pickle

## Expressions des variables de contrôle de premier ordre

Dans cette partie, nous présentons deux méthodes pour calculer les expressions des variables de contrôle de premier ordre utilisées dans la méthode ZV-MCMC.   

L'expression d'une variable de contrôle $ z_i $ avec $ i $ appartenant à $ {1, 2, 3\} est donnée par :
$$
\frac{\partial \ln \pi}{\partial \omega_i} = \frac{\omega_i}{2\sigma^2(\omega_i)} + \frac{1}{4} \sum_{t=1}^{T} \frac{1}{h_t} \frac{\partial h_t}{\partial \omega_i} - \frac{1}{4} \sum_{t=1}^{T} \frac{r_t^2}{h_t^2} \frac{\partial h_t}{\partial \omega_i}, \quad i = 1, 2, 3,
$$


### première méthode

In [ ]:
def calcul_z_1(echant, r, var_emp):
    """
    Calcule les expressions des variables de contrôle z1, z2 et z3.

    Args:
        echant (array): échantillons des paramètres du vecteur omega.
        r (array): Données des rendements.
        variances_empiriques (array): Liste des variances empiriques estimées pour omega_1, omega_2 et omega_3.

    Returns:
        array: une matrice où les variables z_1, z_2, et z_3 sont empilées en colonnes.
    """
    Z1 = 0
    Z2 = 0
    Z3 = 0
    
    # Initialisation des dérivées partielles de h par rapport à omega_1, omega_2 et omega_3
    dht_domega1 = 0  
    dht_domega2 = 0  
    dht_domega3 = 0  

    for i in range(1, len(r)):
        r_t = r[i]    # Rendement à l'instant t
        r_previous = r[i - 1]  # Rendement à l'instant t-1 
        h = 0    
        if i == 1:
            # Calcul de Z1 pour le premier élément de la somme
            Z1 += (1 - echant[2]) / (1 - echant[2]) * (-(0.5 / h) + (0.5 * r_t ** 2) / (h ** 2))
            # Calcul de Z2 et Z3
            Z2 += dht_domega2 * (-(0.5 / h) + (0.5 * r_t ** 2) / (h ** 2))
            Z3 += dht_domega3 * (-(0.5 / h) + (0.5 * r_t ** 2) / (h ** 2))
        else:
            dht_domega1 = r_previous ** 2 + echant[2] * dht_domega1
            dht_domega2 = h + echant[2] * dht_domega2
            dht_domega3 = h + echant[2] * dht_domega3
            # Mise à jour de h
            h = echant[0] + echant[1] * r_previous ** 2 + echant[2] * h
            # Calcul des sommes Z1, Z2 et Z3
            Z1 += dht_domega1 * (-(0.5 / h) + (0.5 * r_t ** 2) / (h ** 2))
            Z2 += dht_domega2 * (-(0.5 / h) + (0.5 * r_t ** 2) / (h ** 2))
            Z3 += dht_domega3 * (-(0.5 / h) + (0.5 * r_t ** 2) / (h ** 2))

    Z1 = -0.5 * (-echant[0] / var_emp[0] + Z1)
    Z2 = -0.5 * (-echant[1] / var_emp[1] + Z2)
    Z3 = -0.5 * (-echant[2] / var_emp[2] + Z3)

    return np.column_stack([Z1, Z2, Z3])


Dans la **deuxième méthode**, nous présentons une fonction qui calcule les $ z_i $ à partir de leur expression théorique, dont le calcul est détaillé dans les diapositives. Les expressions finales sont :

$$
z_1 = \frac{\omega_1}{2\sigma^2(\omega_1)} + \frac{1}{4} \sum_{t=1}^{T} \frac{1}{h_t} \left(\frac{ 1 - \omega_{3}^{t-1}}{1 - \omega_3} \right) - \frac{1}{4} \sum_{t=1}^{T} \frac{r_t^2}{h_t^2} \left(\frac{ 1 - \omega_{3}^{t-1}}{1 - \omega_3}\right)
$$

$$
z_2 = \frac{\omega_2}{2\sigma^2(\omega_2)} + \frac{1}{4} \sum_{t=2}^{T} \frac{1}{h_t} \left(\sum_{i=1}^{t-1} \omega_3^{t-1-i} \cdot r_i^2 \right) - \frac{1}{4} \sum_{t=2}^{T} \frac{r_t^2}{h_t^2} \left(\sum_{i=1}^{t-1} \omega_3^{t-1-i} \cdot r_i^2 \right)
$$

$$
z_3 = \frac{\omega_3}{2\sigma^2(\omega_3)} + \frac{1}{4} \sum_{t=2}^{T} \frac{1}{h_t} \left(\sum_{i=1}^{t-1} \omega_3^{t-1-i} \cdot r_i^2\right) - \frac{1}{4} \sum_{t=2}^{T} \frac{r_t^2}{h_t^2} \left(\sum_{i=1}^{t-1} \omega_3^{t-1-i} \cdot h_i^2 \right)
$$

Ces expressions sont utilisées pour calculer les $ z_i $ dans la méthode ZV-MCMC. Elles sont obtenues en utilisant les expressions théoriques détaillées dans les diapositives.


### deuxième méthode

In [24]:
def calcul_z_2(omega_1, omega_2, omega_3, h, r, var_emp):
    """
    Cette fonction calcule le vecteur des variables de contrôle suivant la première méthode.

    Args:
        omega_1 (array): échantillon du vecteur omega_1.
        omega_2 (array): échantillon du vecteur omega_2.
        omega_3 (array): échantillon du vecteur omega_3.
        h (list) : échantillon des données de volatilité (h_t).
        r (list) : données des rendements logarithmques (r_t).
        var_emp (array): Liste des variances empiriques estimées pour omega_1, omega_2 et omega_3.

    Returns:
        array: une matrice où les variables z_1, z_2, et z_3 sont empilées en colonnes.
    """
    T = len(h)
    z_1 = omega_1 / (2 * var_emp[0] ** 2)
    z_2 = omega_2 / (2 * var_emp[1] ** 2)
    z_3 = omega_3 / (2 * var_emp[2] ** 2)
    
    for t in range(1,T):
        dht_domega1 = (1 - omega_3 ** (t-1)) / (1 - omega_3)
        z_1 += 0.25 * (1/h[t]) * dht_domega1 - int((r[t] ** 2 / h[t]**2)) * dht_domega1
        dht_domega2 = np.array([np.sum([w_i**(t-2-i) * r[i]**2 for i in range(t-1)]) for w_i in omega_2])
        z_2 += 0.25 * (1/h[t]) * dht_domega2 - r[t] ** 2 / h[t]**2 * dht_domega2
        dht_domega3 = np.array([np.sum([w_i**(t-2-i) * h[i]**2 for i in range(t-1)]) for w_i in omega_3])
        z_3 += 0.25 * (1/h[t]) * dht_domega3 - (r[t] ** 2 / h[t]) * dht_domega3
    return np.column_stack([z_1, z_2, z_3])

In [ ]:
# Chargement des variables
with open('donnees.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

y = loaded_data['y']
r = loaded_data['r']
h = loaded_data['h']
echant_sim = loaded_data['echant_sim']
echant_reel = loaded_data['echant_reel']

In [25]:
# Les variables de contrôle pour les données simulées

var_sim = np.var(y[:100], ddof=0)
CV1_sim = calcul_z(echant_sim[:,0], echant_sim[:,1], echant_sim[:,2], h[:100], y[:100], var_sim)


# Les variables de contrôle pour les données réelles 

omega1 = echant_reel[:,0].mean()
omega2 = echant_reel[:,1].mean()
omega3 = echant_reel[:,2].mean()

h_reel = np.zeros(len(r[:100])) 

# On estime la variance conditionnelle des données réelles en estimant les omegas par les moyennes empiriques de l'échantillon généré
for t in range(1,len(r[:100])) :
    h_reel[t] = omega1 + omega2 * h_reel[t-1] + omega3 * r[t-1]**2

var_reel = np.var(r[:100], ddof=0)
CV1_reel = calcul_z(echant_reel[:,0], echant_reel[:,1], echant_reel[:,2], h_reel, r[:100], var_reel)

NameError: name 'y' is not defined

In [ ]:
def enlever_outliers(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)

    IQR = Q3 - Q1

    # Définition des limites
    limite_inferieure = Q1 - 10 * IQR
    limite_superieure = Q3 + 10 * IQR
    donnees_sans_outliers = np.copy(data)
    for i in range(1, len(data)):
        if (data[i] < limite_inferieure).any() or (data[i] > limite_superieure).any():
            donnees_sans_outliers[i] = donnees_sans_outliers[i - 1]
    return donnees_sans_outliers

In [ ]:


def combi_omega(k, echant_sim, CV1_sim, CV1_nv_sim):

    model = sm.OLS(echant_sim[:,k], CV1_sim) # CV1_sim = sm.add_constant(CV1_sim)
    results = model.fit(cov_type='HC3')
    coeff_opt = results.params # CV1_nv_sim1 = np.insert(CV1_nv_sim, 0, 1, axis=1)
    combi = np.dot(coeff_opt, CV1_nv_sim.transpose())
    print(results.summary())
    return combi


# ------------------------------------- Omega_1----------------------------------------------------------------

combi_omega1 = combi_omega(0, echant_sim, CV1_sim, CV1_nv_sim)
omega_tilde_1 = omega_nv_sim[:,0] - combi_omega1
omega_tilde_1ss = enlever_outliers(omega_tilde_1)

# ------------------------------------- Omega_2----------------------------------------------------------------

combi_omega2 = combi_omega(1, echant_sim, CV1_sim, CV1_nv_sim)
omega_tilde_2 = omega_nv_sim[:,1] - combi_omega2
omega_tilde_2ss = enlever_outliers(omega_tilde_2)

# ------------------------------------- Omega_3----------------------------------------------------------------

combi_omega3 = combi_omega(2, echant_sim, CV1_sim, CV1_nv_sim)
omega_tilde_3 = omega_nv_sim[:,2] - combi_omega3
omega_tilde_3ss = enlever_outliers(omega_tilde_3)
